In [1]:
import gc
import pandas as pd
import numpy as np
import os

from glob import glob
from tqdm import tqdm_notebook, tqdm

pd.set_option('display.max_columns', 100)

from utils import loadpkl, to_feature, line_notify
from utils import NUM_FOLDS, FEATS_EXCLUDED, COLS_TEST1, COLS_TEST2
from utils import CustomTimeSeriesSplitter, DAYS_PRED

%matplotlib inline

In [8]:
files = sorted(glob('../feats/*.feather'))
df = pd.concat([pd.read_feather(f) for f in tqdm(files, mininterval=60)], axis=1)

  0%|          | 0/53 [00:00<?, ?it/s]/home/fujiwara/.pyenv/versions/anaconda3-5.2.0/lib/python3.6/site-packages/pandas/io/feather_format.py:124: FutureWarning: `nthreads` argument is deprecated, pass `use_threads` instead
  nthreads=int_use_threads)
/home/fujiwara/.pyenv/versions/anaconda3-5.2.0/lib/python3.6/site-packages/pyarrow/pandas_compat.py:751: FutureWarning: .labels was deprecated in version 0.24.0. Use .codes instead.
  labels, = index.labels
100%|██████████| 53/53 [00:08<00:00,  6.32it/s]


In [6]:
df = df[df['date']>'2014-04-25']

# split train & test
train_df = df[df['date']<'2016-04-25']
test_df = df[df['date']>='2016-04-25']
del df
gc.collect()

22

In [13]:
folds = CustomTimeSeriesSplitter(n_splits=5, train_days=365*2, test_days=DAYS_PRED, day_col='d_numeric')

In [14]:
for n_fold, (train_idx, valid_idx) in enumerate(folds.split(train_df)):
    print(n_fold, (train_idx, valid_idx))

0 (array([24218688, 24218689, 24218690, ..., 41675044, 41675045, 41675046]), array([41675047, 41675048, 41675049, ..., 42528117, 42528118, 42528119]))
1 (array([24218688, 24218689, 24218690, ..., 42528117, 42528118, 42528119]), array([42528120, 42528121, 42528122, ..., 43381454, 43381455, 43381456]))
2 (array([24218688, 24218689, 24218690, ..., 43381454, 43381455, 43381456]), array([43381457, 43381458, 43381459, ..., 44235057, 44235058, 44235059]))
3 (array([24218688, 24218689, 24218690, ..., 44235057, 44235058, 44235059]), array([44235060, 44235061, 44235062, ..., 45088777, 45088778, 45088779]))
4 (array([24218688, 24218689, 24218690, ..., 45088777, 45088778, 45088779]), array([45088780, 45088781, 45088782, ..., 45942497, 45942498, 45942499]))


In [6]:
# set id as index
df.set_index('id', inplace=True)

# sort by date
df.sort_values('date',inplace=True)

In [10]:
df = df[df['date']>'2014-04-25']

# split train & test
#=======================================================================
# 2011-01-29 ~ 2016-04-24 : d_1    ~ d_1913
# 2016-04-25 ~ 2016-05-22 : d_1914 ~ d_1941 (public)
# 2016-05-23 ~ 2016-06-19 : d_1942 ~ d_1969 (private)
#=======================================================================

train_df = df[df['date']<'2016-04-25']
test_df = df[df['date']>='2016-04-25']

del df
gc.collect()

11

In [18]:
train_df['d_numeric'].iloc[0]

1184